<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
from datetime import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# site
site = "tongatapu"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-tongatapu-11122025


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [3]:
# joined_df = gpd.read_file(f"training-data/{version}-training.csv")
# joined_df = joined_df.astype("float32")
# joined_df
joined_df = gpd.read_file("training-data/nm-tongatapu-11122025_postcard_4-training.csv")

In [4]:
joined_df

,LULC_code,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,swir22_swir16,mci,ndci,nbi,ndmi,bsi,awei,tc_wetness,y,x
0,4,0.307,0.1063,0.0852,0.11470000000000001,0.07990727,1.3296604e-07,4.5650013e-06,0.3332,0.3604,...,0.7398237515736467,1.9960988296488944,0.18262206843521728,-0.125985695947185,0.125985695947185,-0.1624184943687018,-0.401925,-0.06630840999999996,-2393675.0,3876145.0
1,2,0.34190000000000004,0.08420000000000001,0.06330000000000001,0.0989,0.15601552,6.5507294e-07,9.853097e-06,0.3326,0.3506,...,0.5863207547169811,2.27478376580173,0.28187633262260126,-0.2345188662213396,0.2345188662213396,-0.320514909564934,-0.48922500000000013,-0.04738885999999999,-2404505.0,3875165.0
2,2,0.21050000000000002,0.07100000000000001,0.0717,0.08410000000000001,0.17074585,1.6380071e-06,1.1550514e-05,0.2695,0.3155,...,0.5847860538827259,1.5850903614457832,0.3032384690873405,-0.05302651325662837,0.05302651325662837,-0.2166415175684414,-0.2900750000000001,-0.031707230000000024,-2404585.0,3875195.0
3,2,0.3322,0.1085,0.0794,0.1169,0.13343513,6.2340496e-07,8.235353e-06,0.3098,0.1713,...,0.6318785578747628,2.0380368098159507,0.20073664825046042,-0.11532650663085443,0.11532650663085443,-0.19895135450043686,-0.480275,-0.06727306,-2404405.0,3875085.0
4,2,0.2817,0.1015,0.07490000000000001,0.10160000000000001,0.1305178,6.9586036e-07,8.470897e-06,0.27290000000000003,0.2862,...,0.6515273311897105,1.9782303370786518,0.16769167691676912,-0.06201696512723843,0.06201696512723843,-0.14995162850693328,-0.4263249999999999,-0.07217224,-2404345.0,3875065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,4,0.2348,0.2107,0.1506,0.18910000000000002,0.047303498,8.1950425e-08,2.7149151e-06,0.2388,0.25630000000000003,...,0.9891373801916934,1.0307287093942055,0.038996579247434444,0.1427528294998174,-0.1427528294998174,0.1489455669647786,-0.12095,-0.05396919000000003,-2392605.0,3873985.0
878,4,0.3164,0.2806,0.2096,0.24730000000000002,0.07955737,5.7399273e-08,3.3720999e-06,0.36150000000000004,0.2947,...,0.8916192446973616,0.9110279297437375,0.1062271062271062,0.0998577524893314,-0.0998577524893314,0.08625032571875267,-0.1404749999999999,-0.002636689999999983,-2392785.0,3874015.0
879,4,0.2451,0.218,0.1563,0.1883,0.046965223,1.19036436e-07,2.5377847e-06,0.2389,0.2742,...,0.9691235059760955,1.0830755634114009,0.018681071348188163,0.10269082921471721,-0.10269082921471721,0.11906068254142438,-0.119425,-0.0379753,-2392715.0,3873725.0
880,4,0.33330000000000004,0.27490000000000003,0.1864,0.2248,0.14836566,7.6683756e-07,5.660991e-06,0.3386,0.3295,...,0.9175767030681227,1.0360584395399441,0.07844451894066377,0.0714584203928123,-0.0714584203928123,0.09420479302832242,-0.2402249999999998,-0.03062745999999999,-2392255.0,3873875.0


In [5]:
joined_df=joined_df.drop(columns=["y", "x"])

In [6]:
print(len(joined_df.columns))
joined_df.columns

33


Index(['LULC_code', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'swir22_swir16', 'mci', 'ndci', 'nbi', 'ndmi', 'bsi',
       'awei', 'tc_wetness'],
      dtype='object')

In [7]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(joined_df, test_size=0.2, random_state=1337)

# The classes are the first column
classes = np.array(training_data)[:, 0]

# The observation data is everything after the second column
observations = np.array(training_data)[:, 1:]

# Create a model...
classifier = RandomForestClassifier(max_depth=4)

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [8]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

['models/nm-tongatapu-11122025-test.model']

In [9]:
import pandas as pd
test_actual = np.array(test_data)[:, 0]

test_predicted = model.predict(np.array(test_data)[:,1:])


unique_labels = sorted(np.unique(np.concatenate([np.asarray(test_actual), np.asarray(test_predicted)])))


pd.crosstab(test_actual, test_predicted, margins=True)

col_0,1,2,3,4,5,6,All
row_0,,,,,,,
1,32,6,1,0,0,0,39
2,5,54,0,3,2,0,64
3,1,2,11,0,0,0,14
4,1,4,0,29,2,0,36
5,1,3,0,9,5,0,18
6,0,0,0,0,1,5,6
All,40,69,12,41,10,5,177


In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(test_actual, test_predicted)

0.768361581920904